# 변종2

* KS통계량 값을 가중치로 학습시키기 
* 1  컬럼수만큼이 좋을지 
* 2  1이라는 수가 좋을지 
* magan 처럼 boundary 를 주는 방식으로

## > 학습이 잘 안된다.....
## > Hinge loss로 변경하기
## SNGAN https://github.com/isr-wang/SNGAN/blob/master/generator.py

# 必必必必必必必
## KS 통계량을 Tensorflow 로 구현을 해서 minimize하는 형식으로 설정해둠
### 과연 잘 될지.
> 먼가 구현이 안되는 상태

In [1]:

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
from scipy.stats import skew
from scipy.stats import kurtosis
slim = tf.contrib.slim

from tensorflow.contrib.distributions import percentile as tf_percent



In [2]:
import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


In [3]:
data = pd.read_csv("./credit44_sc.csv" )

In [4]:
fac_var = []
num_var = []
for i in list(data) : 
    if data[i].nunique() < 15 : 
        print("factor variable : {}, count : {} ".format(i, data[i].nunique()))
        fac_var.append(i)
        if data[i].nunique() == 2 :
            print("binary" , i)
    else : 
        print("연속형 variable : {} , count : {} ".format(i, data[i].nunique()))
        num_var.append(i)
        
data = data.loc[:,fac_var + num_var]

col = list(data)


연속형 variable : SCORE_ORG , count : 395 
연속형 variable : SCORE_NK0200_000 , count : 445 
연속형 variable : SCORE_RK0400_700 , count : 294 
factor variable : CNT_CONTACT_POS, count : 8 
factor variable : CNT_ENG, count : 7 
factor variable : DAYS_CONTACT_POS, count : 11 
factor variable : DAYS_CALL_PAYMENT, count : 11 
factor variable : EWS_C_N_P27000100, count : 14 
연속형 variable : EWS_A_K_D1M232000_OPR , count : 2384 
factor variable : EWS_C_N_P42000200, count : 14 
연속형 variable : EWS_C_N_P32003000 , count : 31 
연속형 variable : AGE , count : 54 
연속형 variable : CNT_Contact , count : 29 
연속형 variable : EWS_A_K_D1M23200C_OPR , count : 2384 
연속형 variable : EWS_C_K_D10310000_OPR , count : 52 
연속형 variable : EWS_A_K_D90232200_OPR , count : 2490 
연속형 variable : D_N_CA0000603 , count : 4894 
연속형 variable : EWS_C_N_P32002600 , count : 22 
연속형 variable : EWS_D_N_P43004000 , count : 399 
factor variable : C_N_PS0001777, count : 11 
factor variable : A_K_D10220000_OPR, count : 9 
연속형 variable : SCORE_RK

In [ ]:

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def mse_mean( real , fake ) : 
    first = log(real) - log(tf.keras.backend.mean(fake , axis = 0 ))
    second = tf.square(first)
    third = tf.reduce_mean(second)
    return third

def mse_std( real , fake ) : 
    first = log(real) -  log(tf.keras.backend.std(fake , axis = 0 ))
    second = tf.square(first)
    third = tf.reduce_mean(second)
    return third

def percentile( real , fake , q=25.0) : 
    first  = log(real) - log(tf.expand_dims(tf_percent(fake , axis = 0 , q= q) , axis = 0))
    second = tf.square(first)
    third  = tf.reduce_mean(second)
    return third


def log(x):
    return tf.log( tf.maximum(x , 1e-8) )


def spectral_norm(w, iteration=1 , name = None):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])
    
    u = tf.get_variable(name , [1, w_shape[-1]], initializer=tf.random_normal_initializer(), trainable=False)

    u_hat = u
    v_hat = None
    for i in range(iteration):
       
        """
       power iteration
       Usually iteration = 1 will be enough
       """
        
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    u_hat = tf.stop_gradient(u_hat)
    v_hat = tf.stop_gradient(v_hat)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = w / sigma
        w_norm = tf.reshape(w_norm, w_shape)

    return w_norm 

# =====================
# Tuning Path , Hidden , etc
# =====================

In [ ]:
mb_size = 5000
X_dim = np.shape(data)[1]
z_dim = 64
h_dim = 128
lr = 1e-5
d_steps = 3
path_v = "V5"

mb_size = 32
X_dim = np.shape(data)[1]
z_dim = 64
h_dim = 128
lr = 1e-3
d_steps = 3

나쁘지 않게 나온다.

In [ ]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, X_dim], name="X")
z = tf.placeholder(tf.float32, shape=[None, z_dim], name="z")

m = tf.placeholder(tf.float32 , name  = "KS")


Hint_mean = tf.placeholder(tf.float32 , [None , X_dim] , name = "mean")
Hint_std = tf.placeholder(tf.float32 , [None , X_dim] , name = "std")

Hint_q25 = tf.placeholder(tf.float32 , [None , X_dim] , name = "q25")
Hint_q75 = tf.placeholder(tf.float32 , [None , X_dim] , name = "q75")



def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

device = ['/cpu:0' , '/cpu:0']


def D(X,reuse=tf.AUTO_REUSE ):
    with tf.device(device[0]) :
        with tf.variable_scope( "Discriminator" ,reuse=reuse  ) as scope :
            D_W1 = tf.get_variable("DW1",[X_dim, h_dim], dtype=tf.float32)
            D_b1 = tf.get_variable("DB1", [h_dim], dtype=tf.float32 , initializer=tf.constant_initializer(0.0))
            D_W2 = tf.get_variable("DW2",[h_dim, h_dim/2], dtype=tf.float32)
            D_b2 = tf.get_variable("DB2",[h_dim/2], dtype=tf.float32, initializer=tf.constant_initializer(0.0))
            D_W3 = tf.get_variable("DW3",[h_dim/2, h_dim/4], dtype=tf.float32)
            D_b3 = tf.get_variable("DB3",[h_dim/4], dtype=tf.float32, initializer=tf.constant_initializer(0.0))
            D_W4 = tf.get_variable("DW4",[h_dim/4, h_dim/8], dtype=tf.float32)
            D_b4 = tf.get_variable("DB4",[h_dim/8], dtype=tf.float32, initializer=tf.constant_initializer(0.0))
            D_W5 = tf.get_variable("DW5",[h_dim/8, 1], dtype=tf.float32)
            D_b5 = tf.get_variable("DB5",[1], dtype=tf.float32, initializer=tf.constant_initializer(0.0))
            
            D_h1 = tf.nn.leaky_relu(tf.matmul(X, spectral_norm(D_W1 , name ="sn1")) + D_b1)
            D_h2 = tf.nn.leaky_relu(tf.matmul(D_h1, spectral_norm(D_W2 , name ="sn2")) + D_b2)
            D_h3 = tf.nn.leaky_relu(tf.matmul(D_h2, spectral_norm(D_W3 , name ="sn3")) + D_b3)
            D_h4 = tf.nn.leaky_relu(tf.matmul(D_h3, spectral_norm(D_W4 , name ="sn4")) + D_b4)
            out = tf.matmul(D_h4, D_W5) + D_b5
        return out


def G(X = None , Z = None , reuse = tf.AUTO_REUSE) : 
    with tf.device(device[1]) :
        with tf.variable_scope("Generator" , reuse = reuse) as scope: 
            G_W1 = tf.get_variable("GW1",[z_dim, h_dim], dtype=tf.float32 , initializer=tf.contrib.layers.xavier_initializer())
            G_b1 = tf.get_variable("GB1",[h_dim],dtype=tf.float32, initializer=tf.constant_initializer(0.0))
            G_W2 = tf.get_variable("GW2" ,[h_dim, h_dim/2], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())
            G_b2 = tf.get_variable("GB2",[h_dim/2], dtype=tf.float32, initializer=tf.constant_initializer(0.0))
            G_W3 = tf.get_variable("GW3" ,[h_dim/2, h_dim/4], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())
            G_b3 = tf.get_variable("GB3",[h_dim/4], dtype=tf.float32, initializer=tf.constant_initializer(0.0))
            G_W4 = tf.get_variable("GW4" ,[h_dim/4, X_dim], dtype=tf.float32)
            G_b4 = tf.get_variable("GB4",[X_dim], dtype=tf.float32, initializer=tf.constant_initializer(0.0))
            
            G_h1 = tf.matmul(Z, spectral_norm(G_W1 , name="Gsn1") ) + G_b1
            G_h1 = tf.layers.dropout(G_h1)
            G_h1 = tf.nn.leaky_relu(G_h1)
            G_h2 = tf.matmul(G_h1, spectral_norm(G_W2 , name="Gsn2")) + G_b2
            #G_h2 = tf.layers.dropout(G_h2)
            G_h2 = tf.nn.leaky_relu(G_h2)
            G_h3 = tf.matmul(G_h2, spectral_norm(G_W3, name="Gsn3")) + G_b3
            #G_h3 = tf.layers.dropout(G_h3)
            G_h3 = tf.nn.leaky_relu(G_h3)
            
            G_log_prob = tf.matmul(G_h3, G_W4) # + G_b2 Bias는 일단 제외 
            G_log_prob = tf.nn.sigmoid(G_log_prob)
            
#             one = tf.slice(G_log_prob , [0, 0] , [1,-1] )[0]
#             two = tf.slice(G_log_prob , [0, 1] , [1,-1] )[0]
#             thr = tf.slice(G_log_prob , [0, 2] , [1,-1] )[0]
#             fou = tf.slice(G_log_prob , [0, 3] , [1,-1] )[0]
#             fif = tf.slice(G_log_prob , [0, 4] , [1,-1] )[0]
#             six = tf.slice(G_log_prob , [0, 5] , [1,-1] )[0]
#             sev = tf.slice(G_log_prob , [0, 6] , [1,-1] )[0]
#             eig = tf.slice(G_log_prob , [0, 7] , [1,-1] )[0]
#             nin = tf.slice(G_log_prob , [0, 8] , [1,-1] )[0]
#             ten = tf.slice(G_log_prob , [0, 9] , [1,-1] )[0]
#             one1 = tf.slice(G_log_prob , [0, 10] , [1,-1] )[0]
#             one2 = tf.slice(G_log_prob , [0, 11] , [1,-1] )[0]
#             one3 = tf.slice(G_log_prob , [0, 12] , [1,-1] )[0]
#             one4 = tf.slice(G_log_prob , [0, 13] , [1,-1] )[0]
#             one5 = tf.slice(G_log_prob , [0, 14] , [1,-1] )[0]
#             one6 = tf.slice(G_log_prob , [0, 15] , [1,-1] )[0]
#             one7 = tf.slice(G_log_prob , [0, 16] , [1,-1] )[0]
#             one8 = tf.slice(G_log_prob , [0, 17] , [1,-1] )[0]
#             one9 = tf.slice(G_log_prob , [0, 18] , [1,-1] )[0]
#             two0 = tf.slice(G_log_prob , [0, 19] , [1,-1] )[0]
#             two1 = tf.slice(G_log_prob , [0, 20] , [1,-1] )[0]
#             two2 = tf.slice(G_log_prob , [0, 21] , [1,-1] )[0]
#             two3 = tf.slice(G_log_prob , [0, 22] , [1,-1] )[0]
#             two4 = tf.slice(G_log_prob , [0, 23] , [1,-1] )[0]
#             two5 = tf.slice(G_log_prob , [0, 24] , [1,-1] )[0]
#             two6 = tf.slice(G_log_prob , [0, 25] , [1,-1] )[0]
#             two7 = tf.slice(G_log_prob , [0, 26] , [1,-1] )[0]
#             two8 = tf.slice(G_log_prob , [0, 27] , [1,-1] )[0]
#             two9 = tf.slice(G_log_prob , [0, 28] , [1,-1] )[0]
#             thr0 = tf.slice(G_log_prob , [0, 29] , [1,-1] )[0]
#             thr1 = tf.slice(G_log_prob , [0, 30] , [1,-1] )[0]
#             thr2 = tf.slice(G_log_prob , [0, 31] , [1,-1] )[0]
#             thr3 = tf.slice(G_log_prob , [0, 32] , [1,-1] )[0]
#             thr4 = tf.slice(G_log_prob , [0, 33] , [1,-1] )[0]
#             thr5 = tf.slice(G_log_prob , [0, 34] , [1,-1] )[0]
#             thr6 = tf.slice(G_log_prob , [0, 35] , [1,-1] )[0]
#             thr7 = tf.slice(G_log_prob , [0, 36] , [1,-1] )[0]
#             thr8 = tf.slice(G_log_prob , [0, 37] , [1,-1] )[0]
#             thr9 = tf.slice(G_log_prob , [0, 38] , [1,-1] )[0]
#             fou0 = tf.slice(G_log_prob , [0, 39] , [1,-1] )[0]
#             fou1 = tf.slice(G_log_prob , [0, 40] , [1,-1] )[0]
#             fou2 = tf.slice(G_log_prob , [0, 41] , [1,-1] )[0]
#             fou3 = tf.slice(G_log_prob , [0, 42] , [1,-1] )[0]
#             fou4 = tf.slice(G_log_prob , [0, 43] , [1,-1] )[0]
#             fou5 = tf.slice(G_log_prob , [0, 44] , [1,-1] )[0]
            
            
            one = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 0))
            two = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 1))
            thr = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 2))
            fou = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 3))
            fif = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 4))
            six = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 5))
            sev = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 6))
            eig = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 7))
            nin = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 8))
            ten = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 9))
            one1 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 10))
            one2 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 11))
            one3 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 12))
            one4 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 13))
            one5 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 14))
            one6 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 15))
            one7 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 16))
            one8 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 17))
            one9 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 18))
            two0 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 19))
            two1 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 20))
            two2 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 21))
            two3 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 22))
            two4 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 23))
            two5 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 24))
            two6 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 25))
            two7 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 26))
            two8 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 27))
            two9 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 28))
            thr0 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 29))
            thr1 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 30))
            thr2 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 31))
            thr3 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 32))
            thr4 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 33))
            thr5 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 34))
            thr6 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 35))
            thr7 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 36))
            thr8 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 37))
            thr9 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 38))
            fou0 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 39))
            fou1 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 40))
            fou2 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 41))
            fou3 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 42))
            fou4 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 43))
            fou5 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_log_prob), 44))
            
            
            slice_collection = [one , two , thr , fou , fif , six , sev , eig ,nin , ten , 
                                one1 ,one2 ,one3 , one4 , one5 , one6 ,one7 , one8,one9 , 
                                two0 , two1 ,two2 ,two3 , two4 , two5 , two6 ,two7 , two8,two9 , 
                                thr0 , thr1 ,thr2 ,thr3 , thr4 , thr5 , thr6 ,thr7 , thr8,thr9 ,
                                fou0 , fou1 , fou2 , fou3 , fou4 , fou5
                               ]
        return G_log_prob , slice_collection


def gradient_penalty(real, fake, f ):
    alpha = tf.random_uniform(
        shape=[mb_size ,1], 
        minval=0.,
        maxval=1.
    )
    differences = fake- real
    interpolates = real + (alpha*differences)
    gradients = tf.gradients(f(interpolates), [interpolates])[0]
    slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
    gp = tf.reduce_mean((slopes-1.)**2)
    return gp , gradients

def ks_2samp_tf(G_sample , X , batch_size , index = 0 ):
    
    G_sample2 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_sample), index))
    X2        = tf.transpose(tf.nn.embedding_lookup(tf.transpose(X), index))
    G_sample2 = G_sample
    X2        = tf.slice(X , [0, index] , [1,-1] )[0]
    
    n1        = batch_size
    n2        = batch_size
    data1     = tf.contrib.framework.sort(G_sample2, axis = 0 )
    data2     = tf.contrib.framework.sort(X2 , axis = 0 )
#     data1     =  tf.to_float( data1 ) 
#     data2     =  tf.to_float( data2 ) 
    data_all  = tf.concat([data1,data2] , axis = 0)
    cdf1      = tf.searchsorted(data1,data_all,side='right' ) / ( mb_size)
    cdf2      = tf.searchsorted(data2,data_all,side='right')/ ( mb_size)
    d         = tf.reduce_max(tf.abs(cdf1-cdf2))
    d         = tf.to_float(d)
    return d

def ks_2samp_tf(G_sample , X , batch_size , index = 0 ):
    
    G_sample2 = G_sample[index]
    X2        = tf.transpose(tf.nn.embedding_lookup(tf.transpose(X), index))
#     G_log_prob2 = G_log_prob
#     X2        = tf.slice(X , [0, index] , [1,-1] )[0]
    
    n1        = batch_size
    n2        = batch_size
    data1     = tf.contrib.framework.sort(G_sample2, axis = 0 )
    data2     = tf.contrib.framework.sort(X2 , axis = 0 )
    data_all  = tf.concat([data1,data2] , axis = 0)
    cdf1      = tf.searchsorted(data1,data_all,side='right' ) / ( mb_size)
    cdf2      = tf.searchsorted(data2,data_all,side='right')/ ( mb_size)
    d         = tf.reduce_max(tf.abs(cdf1-cdf2))
    d         = tf.reduce_mean(tf.square(data1-data2))
    d         = tf.to_float(d)
    return d

def sample_data(data = None , n= len(data)) : 
    output = data[list(np.random.choice(len(data) , n))]
    return output

In [ ]:
data1 = [1,2,3,4,5]
data2 = [6,2,8,1,10]
data_all = np.concatenate([data1 ,data2 ])

print(data_all)
cdf1 = np.searchsorted(data1 , data_all ) / len(data1)
print(cdf1)
cdf2 = np.searchsorted(data2 , data_all) / len(data2)
print(cdf2)
d         = np.max(np.abs(cdf1-cdf2)) 
print(d)

[ 1  2  3  4  5  6  2  8  1 10]
[0.  0.2 0.4 0.6 0.8 1.  0.2 1.  0.  1. ]
[0.  0.  0.4 0.8 0.8 0.8 0.  0.4 0.  0.8]
0.6


In [ ]:
# import tensorflow as tf , numpy as np
# tf.reset_default_graph()
# def loss_func(G , X , batch_size , index = 0 ):  
#     G2 = G[index]
#     X2        = tf.transpose(tf.nn.embedding_lookup(tf.transpose(X), index))    
#     n1 ,n2    = batch_size , batch_size
#     data1     = tf.contrib.framework.sort(G2, axis = 0 )
#     data2     = tf.contrib.framework.sort(X2 , axis = 0 )
#     data_all  = tf.concat([data1,data2] , axis = 0)
#     cdf1      = tf.searchsorted(data1,data_all,side='right' ) / ( batch_size)
#     cdf2      = tf.searchsorted(data2,data_all,side='right')/ ( batch_size)
#     #d         = tf.reduce_max(tf.abs(cdf1-cdf2))
#     d         = tf.reduce_mean(tf.square(data1-data2))
#     return tf.to_float(d)

# X_dim = 45
# z_dim = 30
# h_dim = 45
# batch_size = 500
# X = tf.placeholder(tf.float32, shape=[None, X_dim], name="X")
# z = tf.placeholder(tf.float32, shape=[None, z_dim], name="z")
# D_W1 = tf.get_variable("DW1",[z_dim, h_dim], dtype=tf.float32)
# slice_output = tf.matmul(z , D_W1)
# print(slice_0)
# slice_0 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(slice_output), 0))
# slice_1 = tf.transpose(tf.nn.embedding_lookup(tf.transpose(slice_output), 1))
# slice_output = [slice_0 , slice_1]

# loss1 = loss_func(slice_output , X , batch_size , index = 0 )
# loss2 = loss_func(slice_output , X , batch_size , index = 1 )
# loss  = loss1 + loss2
# # sess = tf.InteractiveSession()
# X11 = np.random.normal(size= [1000,45])
# Z11 = np.random.normal(size= [1000,30])
# # H = np.random.normal(size= [1000,45])
# # H =[tf.transpose(tf.nn.embedding_lookup(tf.transpose(H), 0))]
# # index = 0
# # mb_size = 500
# # X11 = np.random.normal(size= [1000,45])
# # H2 = loss_func(H ,X11 , mb_size , index)
# # print(H2.eval())


# opt = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for i in range(10000):
#         _ , lss = sess.run([opt, loss] , feed_dict={z: Z11 , X : X11})
#         if i % 1000  == 0:
#             print(lss)


In [ ]:
def ks_2samp_tf_total(G_sample , X , mb_size ):
    ks_tf0 = ks_2samp_tf(G_sample ,X , mb_size , 0)
    ks_tf1 = ks_2samp_tf(G_sample ,X , mb_size , 1)
    ks_tf2 = ks_2samp_tf(G_sample ,X , mb_size , 2)
    ks_tf3 = ks_2samp_tf(G_sample ,X , mb_size , 3)
    ks_tf4 = ks_2samp_tf(G_sample ,X , mb_size , 4)
    ks_tf5 = ks_2samp_tf(G_sample ,X , mb_size , 5)
    ks_tf6 = ks_2samp_tf(G_sample ,X , mb_size , 6)
    ks_tf7 = ks_2samp_tf(G_sample ,X , mb_size , 7)
    ks_tf8 = ks_2samp_tf(G_sample ,X , mb_size , 8)
    ks_tf9 = ks_2samp_tf(G_sample ,X , mb_size , 9)
    ks_tf10 = ks_2samp_tf(G_sample ,X , mb_size , 10)
    ks_tf11 = ks_2samp_tf(G_sample ,X , mb_size , 11)
    ks_tf12 = ks_2samp_tf(G_sample ,X , mb_size , 12)
    ks_tf13 = ks_2samp_tf(G_sample ,X , mb_size , 13)
    ks_tf14 = ks_2samp_tf(G_sample ,X , mb_size , 14)
    ks_tf15 = ks_2samp_tf(G_sample ,X , mb_size , 15)
    ks_tf16 = ks_2samp_tf(G_sample ,X , mb_size , 16)
    ks_tf17 = ks_2samp_tf(G_sample ,X , mb_size , 17)
    ks_tf18 = ks_2samp_tf(G_sample ,X , mb_size , 18)
    ks_tf19 = ks_2samp_tf(G_sample ,X , mb_size , 19)
    ks_tf20 = ks_2samp_tf(G_sample ,X , mb_size , 20)
    ks_tf21 = ks_2samp_tf(G_sample ,X , mb_size , 21)
    ks_tf22 = ks_2samp_tf(G_sample ,X , mb_size , 22)
    ks_tf23 = ks_2samp_tf(G_sample ,X , mb_size , 23)
    ks_tf24 = ks_2samp_tf(G_sample ,X , mb_size , 24)
    ks_tf25 = ks_2samp_tf(G_sample ,X , mb_size , 25)
    ks_tf26 = ks_2samp_tf(G_sample ,X , mb_size , 26)
    ks_tf27 = ks_2samp_tf(G_sample ,X , mb_size , 27)
    ks_tf28 = ks_2samp_tf(G_sample ,X , mb_size , 28)
    ks_tf29 = ks_2samp_tf(G_sample ,X , mb_size , 29)
    ks_tf30 = ks_2samp_tf(G_sample ,X , mb_size , 30)
    ks_tf31 = ks_2samp_tf(G_sample ,X , mb_size , 31)
    ks_tf32 = ks_2samp_tf(G_sample ,X , mb_size , 32)
    ks_tf33 = ks_2samp_tf(G_sample ,X , mb_size , 33)
    ks_tf34 = ks_2samp_tf(G_sample ,X , mb_size , 34)
    ks_tf35 = ks_2samp_tf(G_sample ,X , mb_size , 35)
    ks_tf36 = ks_2samp_tf(G_sample ,X , mb_size , 36)
    ks_tf37 = ks_2samp_tf(G_sample ,X , mb_size , 37)
    ks_tf38 = ks_2samp_tf(G_sample ,X , mb_size , 38)
    ks_tf39 = ks_2samp_tf(G_sample ,X , mb_size , 39)
    ks_tf40 = ks_2samp_tf(G_sample ,X , mb_size , 40)
    ks_tf41 = ks_2samp_tf(G_sample ,X , mb_size , 41)
    ks_tf42 = ks_2samp_tf(G_sample ,X , mb_size , 42)
    ks_tf43 = ks_2samp_tf(G_sample ,X , mb_size , 43)
    ks_tf44 = ks_2samp_tf(G_sample ,X , mb_size , 44)
    stat_ks_total = tf.reduce_sum([ks_tf0 , ks_tf1 , ks_tf2 , ks_tf3 , ks_tf4, ks_tf5 , ks_tf6 , ks_tf7 , ks_tf8 , ks_tf9,
                                   ks_tf10 , ks_tf11 , ks_tf12 , ks_tf13 , ks_tf14,ks_tf15 , ks_tf16 , ks_tf17 , ks_tf18 , ks_tf19,
                                   ks_tf20 , ks_tf21 , ks_tf22 , ks_tf23 , ks_tf24,ks_tf25 , ks_tf26 , ks_tf27 , ks_tf28 , ks_tf29,
                                   ks_tf30 , ks_tf31 , ks_tf32 , ks_tf33 , ks_tf34,ks_tf35 , ks_tf36 , ks_tf37 , ks_tf38 , ks_tf39,
                                   ks_tf40 , ks_tf41 , ks_tf42 , ks_tf43 , ks_tf44] , name = "STACK_KS")
    #stat_ks_total = stat_ks_total # , tf.float32 , name="total_ks_cast")
    return stat_ks_total



In [ ]:
# sess = tf.InteractiveSession()
# G_sample = np.random.normal(size= [1000,45])
# G_sample =[tf.transpose(tf.nn.embedding_lookup(tf.transpose(G_sample), 0))]
# X11 = np.random.normal(size= [1000,45])
# index = 0
# mb_size = 500
# G_sample2 = ks_2samp_tf(G_sample ,X11 , mb_size , index)
# G_sample2.eval()
# tf.slice(G_sample , [0, index] , [1,-1] ).eval()
# tf.slice(G_sample , [0, index] , [1,-1] ).eval()
# sess.close()

In [ ]:
G_sample , ks= G(X , z)

D_real = D( X )
D_fake = D( G_sample )


#Z = tf.reduce_sum(tf.exp(-D_real)) + tf.reduce_sum(tf.exp(-D_fake))
# D_loss = -tf.reduce_mean(log(D_real) + log(1 - D_fake))
# G_loss = 0.5 * tf.reduce_mean((log(D_fake) - log(1 - D_fake))**2)



fake_logit = D_fake
real_logit = D_real
#D_loss = - (tf.reduce_mean(log(real_logit ) + log(1 - fake_logit )))


"""
1 . wgan loss는 0으로 갈수록 좋음 그러므로 0으로 가면서 ks 통계량 값을(45) 를 그대로 주는 방식으로 해보기 
    물론 너무 크게 학습이 될 우려가 아주 심하지만.... 음....
    결국 그 자체 값의 가중치 주기
    HINGE LOSS 방식으로 진행.

2. Boundart
"""

### Hinge Loss
# real_loss = tf.reduce_mean( tf.nn.relu(1.0 - real_logit) )
# fake_loss = tf.reduce_mean( tf.nn.relu(1.0 + fake_logit) )
# D_loss = real_loss + fake_loss

# Gp , gradient_check = gradient_penalty(X, G_sample , D )




########### Boundary loss
D_real2 = tf.nn.sigmoid(real_logit)
D_fake2 = tf.nn.sigmoid(fake_logit)
G_loss = 0.5 * tf.reduce_mean((log(D_fake2) - log(1 - D_fake2))**2) + ks_2samp_tf_total(ks , X , mb_size)
D_loss = -tf.reduce_mean(log(D_real2) + log(1 - D_fake2)) 


q75_loss = percentile(Hint_q25 , G_sample , q=75.0) * 0.01
q25_loss = percentile(Hint_q75 , G_sample , q=25.0) * 0.01

mean_loss = mse_mean(Hint_mean , G_sample)
std_loss  = mse_std( Hint_std , G_sample)

#G_recon_loss = 
# + ks_tf
# ks_tf = tf.cast(ks_tf , tf.float32)
# stat_ks_total = tf.reduce_sum([ks_tf0 , ks_tf1 , ks_tf2 , ks_tf3 , ks_tf4,
#                      ks_tf5 , ks_tf6 , ks_tf7 , ks_tf8 , ks_tf9,
#                      ks_tf10 , ks_tf11 , ks_tf12 , ks_tf13 , ks_tf14,
#                      ks_tf15 , ks_tf16 , ks_tf17 , ks_tf18 , ks_tf19,
#                      ks_tf20 , ks_tf21 , ks_tf22 , ks_tf23 , ks_tf24,
#                      ks_tf25 , ks_tf26 , ks_tf27 , ks_tf28 , ks_tf29,
#                      ks_tf30 , ks_tf31 , ks_tf32 , ks_tf33 , ks_tf34,
#                      ks_tf35 , ks_tf36 , ks_tf37 , ks_tf38 , ks_tf39,
#                      ks_tf40 , ks_tf41 , ks_tf42 , ks_tf43 , ks_tf44] , name = "STACK_KS")

# #stat_ks_total = tf.reduce_sum(stack_ks, name="total_ks")
#stat_ks_total = ks_2samp_tf_total(G_sample ,X , mb_size )
#stat_ks_total = tf.cast(ks_2samp_tf(G_sample ,X , mb_size , 0) , tf.float32)
#stat_ks_total = ks_2samp_tf(G_sample ,X , mb_size , 0) 

#G_recon_loss = q75_loss + q25_loss + mean_loss + std_loss 

ind = tf.constant([0, 2])

result = tf.transpose(tf.nn.embedding_lookup(tf.transpose(a), ind))
result.eval(session = sess)
a = np.random.normal(loc=0 ,scale= 1 ,size = [1000, 5])
b = np.random.normal(loc=5 ,scale= 2 ,size = [1000, 5])
c = np.concatenate([a,b])
print(c)
c = tf.searchsorted(c , a  , side='right') / len(a)
d = tf.searchsorted(c , b , side='right') / len(b)
e = tf.reduce_max(tf.abs(c-d) , axis=0)
e.eval(session = sess)
c.eval(session = sess)

config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())
G_sample2 = sample_data(data=data , n=mb_size)
X2 = sample_data(data=data , n=mb_size)
print(ks_2samp_tf(G_sample2 ,X2 , mb_size , 0).eval(session = sess ))
print(ks_2samp_tf(G_sample2 ,X2 , mb_size , 1).eval(session = sess ))
print(ks_2samp_tf(G_sample2 ,X2 , mb_size , 8).eval(session = sess ))

G_sample2 = sample_data(data=data.values , n=mb_size)[:,0]
X2 = sample_data(data=data.values , n=mb_size)[:,0]

data1 = tf.contrib.framework.sort(G_sample2, axis = 0 )
data2 = tf.contrib.framework.sort(X2, axis = 0 )
data_all = tf.concat([data1,data2] , axis = 0)
data_all.eval(session = sess)
cdf1 = tf.searchsorted(data1,data_all,side='right' )/ ( mb_size)
cdf2 = tf.searchsorted(data2,data_all,side='right')/ ( mb_size)
cdf1.eval(session = sess) - cdf2.eval(session = sess)
tf.reduce_max(tf.abs(cdf1-cdf2), axis = 0).eval(session = sess)
cdf2 = tf.searchsorted(data2,data_all,side='right')/ ( mb_size)

In [ ]:
#fake_loss = -tf.reduce_mean(fake)
#D_fake = tf.nn.sigmoid(fake_logit)
#G_loss = 0.5 * tf.reduce_mean((log(D_fake) - log(1 - D_fake))**2)


####################### 새로 배운 것 #########################################

t_vars = tf.trainable_variables()

d_vars = [var for var in t_vars if 'Discriminator' in var.name]
g_vars = [var for var in t_vars if 'Generator' in var.name]
#total_vars =  g_vars + d_vars

print(g_vars, "\n",d_vars)

with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    G_solver = tf.train.AdamOptimizer(lr).minimize(G_loss, var_list=g_vars)
    D_solver = tf.train.AdamOptimizer(lr).minimize(D_loss, var_list=d_vars)
    #G_recon_solver = tf.train.AdamOptimizer(lr).minimize(G_recon_loss, var_list=g_vars)
    
    

# theta_G = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="Generator")
# theta_D = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="Discriminator")


# with tf.variable_scope(tf.get_variable_scope(),reuse=False): 
#     trainerD = tf.train.AdamOptimizer().minimize(D_loss, var_list=d_vars) 
#     trainerG = tf.train.AdamOptimizer().minimize(G_loss, var_list=g_vars)

# D_solver = (tf.train.AdamOptimizer(learning_rate=lr)
#             .minimize(D_loss, var_list=theta_D))

# G_solver = (tf.train.AdamOptimizer(learning_rate=lr)
#             .minimize(G_loss, var_list=theta_G))

[<tf.Variable 'Generator/GW1:0' shape=(64, 128) dtype=float32_ref>, <tf.Variable 'Generator/GB1:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'Generator/GW2:0' shape=(128, 64) dtype=float32_ref>, <tf.Variable 'Generator/GB2:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'Generator/GW3:0' shape=(64, 32) dtype=float32_ref>, <tf.Variable 'Generator/GB3:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'Generator/GW4:0' shape=(32, 45) dtype=float32_ref>, <tf.Variable 'Generator/GB4:0' shape=(45,) dtype=float32_ref>] 
 [<tf.Variable 'Discriminator/DW1:0' shape=(45, 128) dtype=float32_ref>, <tf.Variable 'Discriminator/DB1:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'Discriminator/DW2:0' shape=(128, 64) dtype=float32_ref>, <tf.Variable 'Discriminator/DB2:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'Discriminator/DW3:0' shape=(64, 32) dtype=float32_ref>, <tf.Variable 'Discriminator/DB3:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'Discriminator/DW4:0' shape=(32, 16) dtype=float32

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
def _strip_consts( graph_def):
        from IPython.display import clear_output, Image, display, HTML
        """Strip large constant values from graph_def."""
        strip_def = tf.GraphDef()
        for n0 in graph_def.node:
            n = strip_def.node.add() 
            n.MergeFrom(n0)
            if n.op == 'Const':
                tensor = n.attr['value'].tensor
                size = len(tensor.tensor_content)
        return strip_def


def _show_graph( graph_def):
    from IPython.display import clear_output, Image, display, HTML
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = _strip_consts(graph_def)
    code = """
        <script>
        function load() {{
            document.getElementById("{id}").pbtxt = {data};
        }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
        <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
    iframe = """
        <iframe seamless style="width:100%;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

def tensorboard():
    _show_graph(tf.get_default_graph().as_graph_def())
    
    
tensorboard()

In [ ]:
from sklearn.preprocessing import MinMaxScaler



scaler = MinMaxScaler(feature_range= (0,1))

x_plot = pd.DataFrame(sample_data(data = data.values , n=mb_size) , columns = col)
data = scaler.fit_transform(data)

mean = np.expand_dims(np.mean(data,axis = 0) ,axis = 0)

std = np.expand_dims(np.std(data,axis = 0) ,axis = 0)

q75 = np.expand_dims(np.quantile(data , axis = 0 , q=0.75),axis = 0)

q25 = np.expand_dims(np.quantile(data, axis = 0 , q=0.25),axis = 0)


In [ ]:

import os

def Check_KS(ks_output , Total_ks_graph , Margin , row , ncol) : 
    total = sess.run(G_sample, feed_dict={z: sample_z(mb_size, z_dim)})
    total = scaler.inverse_transform(total)
    g_plot = pd.DataFrame(total , columns = col )
    g_plot[fac_var] = g_plot[fac_var].round(0)
    Value = []
    for label in col :
        sample = g_plot[label]
        real   = x_plot[label]
        ks , p = stats.ks_2samp(real.values , sample.values)
        Value.append(ks)
    Total_KS = round( np.sum(Value) , 2)
    ks_2 = [iteration] + Value
    ks_3 = pd.DataFrame([ks_2], columns = ["iter"] + col)
    ks_output = ks_output.append(ks_3)
    fig , ax = plt.subplots(figsize=(26,13))
    fig.subplots_adjust(top = 0.95 , left = 0.03 , bottom = 0.05 , right = 0.99)
    updown = 0
    for name in col : 
        if updown % 2 == 0 :
            param , space="bottom" , "  "
        else : 
            param , space ="top" , "   "
        ax.plot(ks_output.iter , ks_output[[name]], label = name)
        ax.text(iteration , ks_output.loc[ks_output["iter"]==iteration , [name]].values , space + name ,
                verticalalignment = param)
        updown +=1
    ax.set_title("KS [{}]".format(Total_KS) , fontsize = 30 )
    ax.set_xlabel("iteration")
    ax.set_ylabel("KS")    
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
    ax.set_title("KS Measure {}[<1] , Max : {},  Minimum : {}".format(Total_KS , len(col), Total_ks_graph.ks.min() ) , fontsize = 30)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=15 , fontsize= 10)
    ax.text(iteration+1 , 0.05 , "   0.05", verticalalignment = param)
    ax.axhline(0.05, linewidth=4, color='r')
    plt.savefig("./V0_SN_KS_{}.png".format(path_v))
    plt.show()
    """
    Total Graph
    """
    Total_ks_graph_ap = pd.DataFrame({"iter":[iteration] , "ks" :[Total_KS]})
    Total_ks_graph    = Total_ks_graph.append(Total_ks_graph_ap)
    #Total_ks_graph    = Total_ks_graph[Total_ks_graph.iter>0]
    fig , ax = plt.subplots(figsize = (26,13))
    fig.subplots_adjust(top = 0.95 , left = 0.03 , bottom = 0.04 , right = 0.99)
    ax.plot(Total_ks_graph.iter , Total_ks_graph.ks , linestyle ="-" , marker ="." , linewidth = 3, markersize = 12)
    ax.set_title("Iter : {} , KS : {} , Max : {},  Minimum : {}".format(iteration-1 ,  Total_KS , len(col) , Total_ks_graph.ks.min() ), fontsize= 30)
    plt.savefig("./V0_Total_SN_KS_{}.png".format(path_v))
    plt.show()
    if Margin > Total_KS : 
        show_plot(row , ncol )
        print("=======UPDate============")
        print("Margin : {} ===> {}".format(Margin , Total_KS))
        Margin = Total_KS
        total = sess.run(G_sample, feed_dict={z: sample_z( 5000, z_dim)})
        total = scaler.inverse_transform(total)
        g_plot = pd.DataFrame(total , columns = col )
        g_plot[fac_var] = g_plot[fac_var].round(0)
        g_plot.to_csv("./V0_SN_Generated_{}.csv".format(path_v) , index = False)
        
        saver.save(sess , model_dir + "./V0_SN_MODEL_{}".format(path_v))
    clear_output(wait= True)
    return ks_output , Total_ks_graph , Margin , Total_KS

In [ ]:
from scipy import stats
def show_plot(row , ncol ) : 
    
    fig , axes = plt.subplots(row , ncol , figsize = (26,13))
    fig.subplots_adjust(hspace = 0.2 , wspace= 0.14 , top = 0.92 , left = 0.03 , bottom = 0.04 , right = 0.99)
    total = sess.run(G_sample, feed_dict={z: sample_z(mb_size, z_dim)})
    try : 
        total = total[~np.isnan(total).any(axis=1)]
        total = scaler.inverse_transform(total)
        g_plot = pd.DataFrame(total , columns = col )
        g_plot[fac_var] = g_plot[fac_var].round(0)
        """
        좀 더 쉬운 분포로 만들어서 학습시킨 후 다시 원래값으로 (factor 변수이기 때문에 가능하다 생각함.)
        """

        
        col2 = 0
        error = []
        for j in range(row) :
            for k in range(ncol) :
                try :
                    label = col[col2]
                    sample = g_plot.loc[: , label]
                    sample.name = "Gene"
                    real_0 = x_plot.loc[: , label]
                    real_0.name ="Real"
                    ks , p = stats.ks_2samp(real_0.values , sample.values)
                    error.append(ks)
                    col2 += 1
                    if label in fac_var : 
                        sns.distplot( sample , ax=axes[j , k], norm_hist =True , kde=False , hist_kws ={"color":"r" , "label" :"Gene", "rwidth":0.75})
                        sns.distplot(real_0 , ax=axes[j , k],norm_hist =True, kde=False , hist_kws ={"color":"g" , "label" :"Real", "rwidth":0.75})
                        axes[j , k].legend(fontsize = 10)
                    elif label in num_var : 
                        sns.distplot(  sample , ax=axes[j , k] ,
                                     kde_kws={"color": "r", "lw": 2, "label": "Gene" , "shade" : True} , hist =False , rug = False) #   
                        sns.distplot(  real_0 , ax=axes[j , k] ,
                                     kde_kws={"color": "g", "lw": 2, "label": "Real", "shade" : True } , hist =False , rug = False) # 
                        axes[j , k].legend(fontsize = 10 )
                    axes[j , k].set_title( label , loc ="left" , fontsize= 10 )
                except IndexError as e : 
                    axes[j , k].axis("off")
        
        KS_DIF = round(np.sum(error),2)
        plt.suptitle('SNGAN Iteration {} ,  {} , D_loss : {} , G_loss : {} KS : {}'.format(i,res , dloss,gloss,KS_DIF) , fontsize= 30)
        plt.savefig("./visualization_SN_{}.png".format(path_v))
        plt.show()
        


        fig , ax = plt.subplots(figsize = (26,13))
        fig.subplots_adjust(top = 0.95 , left = 0.03 , bottom = 0.04 , right = 0.99)

        ax.plot(output.iter , output.dloss , label ="dloss" , linestyle ="-" , marker ="." , linewidth = 4, markersize = 12)
        ax.plot(output.iter , output.gloss , label ="gloss" , linestyle ="-" , marker ="." , linewidth = 4, markersize = 12)
        ax.set_title("Iter : {} , Res : {}(>6) , Dloss : {} , Gloss : {}(>2) stat loss : {}".format(iteration-1 ,  res , dloss, gloss , 0), fontsize= 30)
        ax.set_ylim(-5, 15)
        box = ax.get_position()
        ax.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=4 , fontsize= 20)
        plt.savefig("./loss_log_SN_{}.png".format(path_v))
        plt.show()
    
        return print("시각화")

    except Exception as e : 
        print(e)


In [ ]:
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


model_dir = "./Model_Save/{}".format(path_v)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)


config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess = tf.Session(config = config) # config = config
sess.run(tf.global_variables_initializer())


saver = tf.train.Saver()

i = 0
ks_output = pd.DataFrame([[0]+[1.]*len(col)], columns = ["iter"]+col)

iteration , dloss ,  gloss = 0 , 0, 0
output = pd.DataFrame({"iter" : [iteration] , "dloss" : [dloss] , "gloss" : [gloss] })
ks_init = len(col)
Total_ks_graph = pd.DataFrame({"iter":[iteration] , "ks" :[ks_init]})

Margin = 10
ROW , COL = 7,7

print("START")
res ="Boundary Loss Replay Memory"

# for i in range(20000):
#     X_mb = sample_data(data = data , n=mb_size)
#     z_mb = sample_z(mb_size, z_dim)
    
#     _, G_recon_loss_curr = sess.run([G_recon_solver, G_recon_loss ],  # , stat_ks_total # , ks_tf1
#                                     feed_dict={X: X_mb , z: z_mb ,
# #                                                Hint_mean : mean , Hint_std : std , 
# #                                                Hint_q25 : q25 , Hint_q75 : q75  
#                                               })

#     if i % 1000 == 0:
#         clear_output(wait= True)
#         show_plot(row = ROW , ncol= COL)
#         print('Iter-{}; Pretrained G loss: {:.4}'.format(i, G_recon_loss_curr))
#         #print("Q75 :{} ,Q25 : {} , Mean : {} , Std : {}".format( q75_loss1 , q25_loss1 , mean_loss1 , std_loss1))
#         #print("Tensorflow KS : {}".format(ks_tf1))


for i in range(1000000):
    X_mb = sample_data(data = data , n=mb_size)
    z_mb = sample_z(mb_size, z_dim)
    
    
    if i < 200 : 
        for _ in range(10) :
            _, dloss = sess.run(
                [D_solver, D_loss], feed_dict={X: X_mb, z: z_mb})
    else : 
        
        if (Total_KS > Total_ks_graph.ks.min()) & (Total_ks_graph.ks.min() < 9 ): 
            print(" Experience Memory 사용하여 학습 다시 시키기 ")
            chkpt_fname = tf.train.latest_checkpoint( model_dir )
            saver.restore(sess, chkpt_fname)
            z_test_value = sample_z(m=5000 , n=z_dim)
            good_replay_v = sess.run(G_sample, feed_dict={z : z_test_value})
            
            for _ in range(3) :
                _, experience_loss = sess.run(
                    [D_solver, D_loss], feed_dict={X: good_replay_v , z: z_mb })
            
        for _ in range(10) :
            _, dloss = sess.run(
                [D_solver, D_loss], feed_dict={X: X_mb, z: z_mb})
            
    
    for _ in range(5) :
        _, gloss  = sess.run(
                [G_solver, G_loss ], feed_dict={X: X_mb, z: z_mb})

    
    iteration +=1
    
#     D_limit  = 1.0 
#     G_limit  = 1.0 
    
        
#     dcount = 0 
#     while dloss > D_limit  : 
#         dcount += 1
#         _, dloss = sess.run([D_solver, D_loss], feed_dict={X: X_mb, z: z_mb})
#         if dcount > 2000 :
#             print("{}th  Dicriminator Loss : {}".format(i, dloss) )
#             break
#         if dloss < D_limit : 
#             print("D 탈출")
            
#     gcount = 0 
#     while gloss >  G_limit :
#         gcount += 1 
#         _, gloss  = sess.run([G_solver, G_loss ], feed_dict={X: X_mb, z: z_mb})
        
#         if gcount > 1000 :
#             print(" {}th  KS Loss : {}".format(i, gloss) )
#             break
    
#     ks_count = 0 
#     if i > 1000 : 
#         print("G_Limit : {}".format(Total_ks_graph.ks.min()))
#         ks_count += 1 
#         Stat_Loss = Total_KS
#         while Stat_Loss >  Total_ks_graph.ks.min() :
#             _, Stat_Loss = sess.run([G_recon_solver, G_recon_loss  ],feed_dict={X: X_mb , z: z_mb})
#         if ks_count > 1000 : 
#             print("{}th Fail KS Loss : {}".format(i, Stat_Loss) )
#             break
        
        
            
    dloss = round( np.float64(dloss),4)
    gloss = round( np.float64(gloss),4)   
    output1 = pd.DataFrame({"iter" : [iteration] , "dloss" : [dloss] , "gloss" : [gloss] })
    output  = output.append(output1)
    
    
    
    ks_output , Total_ks_graph , Margin  , Total_KS = Check_KS(ks_output , Total_ks_graph , Margin , ROW , COL)
    print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}'
              .format(i, dloss, gloss))
    if (i % 100 == 0) & (Margin < 10.0) :
        clear_output(wait= True)
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}'
              .format(i, dloss, gloss))
        show_plot(row = ROW , ncol= COL)

Iter: 10087; D_loss: 0.6782; G_loss: 1.026


## 실시간 KS 그림을 찍으면서 기준 KS를 통과하면 저장하는 방식으로 진행

## Update
### Tensorflow 에 KS Loss 심음 점점 0으로 가까이가게

## 일단 searchsort가 미분을 할 수 있는 method가 아닌 것 같다.
## 그래도 하나 얻은 점이 있다. 
> ## 특정한 것에 대해서 좀 더 recontrinction loss 로 활용 가능할 듯 
* ### EWS_A_K_D1M232000_OPR  
* ### EWS_A_K_D90232200_OPR
* ### EWS_A_K_D10231000_OPR
* ### EWS_A_K_D10232000_OPR
* ### EWS_C_N_P32002600
* ### Etc